In [1]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4

In [2]:
# User Inputs:
start_time_str       = '2022-02-10T00:00:00Z'
stop_time_str        = '2022-03-01T00:00:01Z'
# plane_callsign       = "EZY158T"
query_limit          = 5e4
send_notification    = True
make_plot            = False
output_dir           = "/scratch/omg28/Data/"

In [ ]:
# Download data from OpenSky history database
download_day(start_time_str, stop_time_str, query_limit, send_notification, make_plot, output_dir)

In [ ]:
load_saved_fd4(start_time_str, output_dir, query_limit)